In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install timm segmentation_models_pytorch rasterio torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=cae7e42862554e10ec5139bef3bc432e192180f4b294193041db58e1b725dac6
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187f

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [18]:
params = {
    'backbone': 'timm-efficientnet-b1',
    'weights': 'imagenet', # 'noisy-student',
    'learning_rate': 0.001,
    'epochs': 50,
    'batch_size': 30,
    'num_workers': 12,
    'device': torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'),
    'div_factor':10,
    'final_div_factor':100,
    'threshhold':0.5
}

## Functions

We define a function on which we can measure the accuracy of our model.

In [5]:
def sort_paths_by_number(paths):
    return sorted(paths, key=lambda x: int(x.split('_')[-1].split('.tif')[0]))

In [6]:
def calculate_f1_score(pred, true, threshold=0.5):
    """
    Compute the F1 score, also known as balanced F-score or F-measure.

    Parameters:
    - true: ground truth (correct) labels, a tensor of shape [N, H, W].
    - pred: predicted labels, a tensor of shape [N, 1, H, W].
    - threshold: the prediction threshold, defaults to 0.5.

    Returns:
    - F1 score
    """

    # Flatten the tensors
    true = true.numpy().flatten()
    pred = pred.numpy().flatten()

    # Calculate F1 score
    f1 = f1_score(true, pred)

    return f1

In [7]:
def intersection_and_union(pred, true):
    """
    Calculates intersection and union for a batch of images.

    Args:
        pred (torch.Tensor): a tensor of predictions
        true (torc.Tensor): a tensor of labels

    Returns:
        intersection (int): total intersection of pixels
        union (int): total union of pixels
    """
    pred = pred.squeeze(1)
    valid_pixel_mask = true.ne(255)  # valid pixel mask
    true = true.masked_select(valid_pixel_mask)
    pred = pred.masked_select(valid_pixel_mask)

    # Convert to CPU and NumPy for logical operations
    true = true.numpy()
    pred = pred.numpy()

    # Intersection and union totals
    intersection = np.logical_and(true, pred)
    union = np.logical_or(true, pred)
    return intersection.sum(), union.sum()

In [9]:
def save_model(epoch,model,ckpt_path='./',name='unet_effnet_b1_1000x1000',val_iou=0):
    path = os.path.join(ckpt_path, '{}_wo_ca.pth'.format(name))
    torch.save(model.state_dict(), path, _use_new_zipfile_serialization=False)

def load_model(model,ckpt_path):
    state = torch.load(ckpt_path)
    model.load_state_dict(state)
    return model

## Loss Functions

In [10]:
class FocalLoss(torch.nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights
        self.bce = torch.nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, input, target):
        #ce_loss = F.binary_cross_entropy_with_logits(input.squeeze(1), target.long(),reduction=self.reduction,weight=self.weight)
        ce_loss = self.bce(input.squeeze(1), target.squeeze(1))
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [11]:
class XEDiceLoss(torch.nn.Module):
    """
    Computes (0.5 * CrossEntropyLoss) + (0.5 * DiceLoss).
    """

    def __init__(self):
        super().__init__()
        self.xe = torch.nn.BCEWithLogitsLoss(reduction="none")
        #self.xe = FocalLoss(reduction='none')

    def forward(self, pred, true):

        pred = pred.squeeze(1)
        valid_pixel_mask = true.ne(255)  # valid pixel mask

        # Cross-entropy loss
        temp_true = torch.where((true == 255), 0, true)  # cast 255 to 0 temporarily
        xe_loss = self.xe(pred, temp_true.float())
        xe_loss = xe_loss.masked_select(valid_pixel_mask).mean()

        # Dice loss

        pred = pred.sigmoid() #torch.softmax(pred, dim=1)[:, 1]

        pred = pred.masked_select(valid_pixel_mask)
        true = true.masked_select(valid_pixel_mask)

        dice_loss = 1 - (2.0 * torch.sum(pred * true) + 1e-7) / (torch.sum(pred + true) + 1e-7)

        #print(xe_loss,dice_loss,(2.0 * torch.sum(pred * true) + 1e-7))

        return (0.5 * xe_loss) + (0.5 * dice_loss)

## Data
We create the Dataset and Dataloader

In [12]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split

import glob
import pandas as pd
import tifffile
import os

In [13]:
PATH_SRC = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/src'
PATH_OUTPUT = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/submit'
PATHS_VAL = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/evaluation_true_color/evaluation_true_color_*.tif'))
PATHS_VAL_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/sample/evaluation_mask_*.tif'))
PATHS_TRAIN = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_*.tif'))
PATHS_TRAIN_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_*.tif'))

In [17]:
class DataTransform():
    def __init__(self):
        self.data_transform = {
            "train": A.Compose(
                [
                  A.ShiftScaleRotate(shift_limit=0.0625,rotate_limit=15,p=0.5),
                  A.GridDistortion(p=0.35),
                  A.HorizontalFlip(p=0.5),
                  A.VerticalFlip(p=0.5),
                  A.GaussianBlur(p=0.25),
                  A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  # A.Resize(512, 512, interpolation=1, p=1),
                  A.RandomCrop(384,384),
                  ToTensorV2()
                ]
            ),
            "val": A.Compose(
                [
                  A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  # A.Resize(512, 512, interpolation=1, p=1),
                  ToTensorV2()
                ]
            )
        }

    def __call__(self, phase, img, mask):
      return self.data_transform[phase](image=img, mask=mask)

In [19]:
class CloudDataset(Dataset):
  def __init__(self, df, phase, transform):
    self.data = df
    self.phase = phase
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def normalize_data(self, image_tiff):
    # Normalize the image for better visualization
    for i in range(3):  # Assuming the image has 3 channels
        image_tiff[:,:,i] = (image_tiff[:,:,i] - image_tiff[:,:,i].min()) / (image_tiff[:,:,i].max() - image_tiff[:,:,i].min())

    return image_tiff

  def __getitem__(self, index):
    row = self.data.iloc[index]
    true_color_PATH = row['true_color']
    mask_PATH = row['mask']
    img = tifffile.imread(true_color_PATH)
    img = self.normalize_data(img.astype(np.float32))
    mask = tifffile.imread(mask_PATH)
    mask = mask.astype(np.float32)

    img_and_mask = self.transform(self.phase, img, mask)
    img = img_and_mask["image"]
    mask = img_and_mask["mask"]

    return img, mask

In [20]:
df = pd.DataFrame({
    'true_color': PATHS_TRAIN,
    'mask': PATHS_TRAIN_MASK
})

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
print(df.head())

df_train, df_val = train_test_split(df, test_size=0.2, random_state=12)

train_dataset = CloudDataset(df_train, phase="train", transform=DataTransform())
train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)


val_dataset = CloudDataset(df_val, phase="val", transform=DataTransform())
val_dataloader = DataLoader(
    val_dataset,
    batch_size=2,#params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)

dataloaders_dict = {
    'train': train_dataloader,
    'val': val_dataloader
}

                                                                                        true_color                                                                                 mask
0  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_0.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_0.tif
1  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_1.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_1.tif
2  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_2.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_2.tif
3  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_3.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_3.tif
4  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/trai

<ipython-input-20-15fac3f26a46>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Model
We implement the cloud detection Model based on the Microsoft Competion Winner: https://github.com/drivendataorg/cloud-cover-winners/blob/main/1st%20place/src/unet_efficient_net_b1_train_fp16.py

In [21]:
from timm.models.efficientnet import *
import segmentation_models_pytorch as smp
import torch.nn as nn
import yaml

In [22]:
class Net(nn.Module):
  def __init__(self,params):
    super(Net, self).__init__()

    aux_params=dict(
        pooling = 'avg',             # one of 'avg', 'max'
        dropout = 0.3,               # dropout ratio, default is None
        activation = None,           # activation function, default is None
        classes = 1,
    )

    self.unet = smp.Unet(
        encoder_name = params['backbone'],        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights = params['weights'],      # use `imagenet` pre-trained weights for encoder initialization
        in_channels = 3,                          # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        decoder_attention_type = None,            # model output channels (number of classes in your dataset)
        classes = 1,
        aux_params = aux_params
    )

  def forward(self, image):
    batch_size = len(image)
    mask,logit = self.unet(image)
    return mask


In [23]:
model = Net(params)
model.to(params['device'])

Downloading: "https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b1-5c1377c4.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b1-5c1377c4.pth
100%|██████████| 30.1M/30.1M [00:02<00:00, 10.9MB/s]


Net(
  (unet): Unet(
    (encoder): EfficientNetEncoder(
      (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNormAct2d(
        32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): Swish()
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (bn1): BatchNormAct2d(
              32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Swish()
            )
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (act1): Swish()
              (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (gate): Sigmoid()
            )
            (conv_pw): C

## Training

In [24]:
from tqdm import tqdm
import matplotlib.pyplot as plt

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

scheduler = torch.optim.lr_scheduler.OneCycleLR(
      optimizer,
      steps_per_epoch = 42,
      pct_start = 0.1,
      max_lr = params['learning_rate'],
      epochs = params['epochs'],
      div_factor = params['div_factor'],
      final_div_factor = params['final_div_factor'],
      verbose = True
    )

criterion = XEDiceLoss()

Adjusting learning rate of group 0 to 1.0000e-04.


In [26]:
def train(model, img, mask):
  model.train()
  optimizer.zero_grad()

  preds = model(img)
  loss = criterion(preds, mask)
  # loss = criterion(preds.squeeze(dim=1), mask.squeeze(dim=1))
  loss.backward()
  optimizer.step()
  loss = loss.item()

  # Calculate validation IOU (global)
  preds = (preds.detach().sigmoid().cpu() > params['threshhold']) *1
  mask = mask.detach().cpu()

  # plt.imshow(preds[0][0], cmap='gray')
  # plt.colorbar()
  # plt.title("pred")
  # plt.show()

  # plt.imshow(mask[0], cmap='gray')
  # plt.colorbar()
  # plt.title("mask")
  # plt.show()


  intersection, union = intersection_and_union(preds, mask)
  f1_score = calculate_f1_score(preds, mask)

  return loss, intersection, union, f1_score

In [27]:
def val(model, img, mask):
  model.eval()
  with torch.no_grad():
    preds = model(img)
    loss = criterion(preds, mask)
    # loss = criterion(preds.squeeze(dim=1), mask.squeeze(dim=1))
    loss = loss.item()

    preds = (preds.detach().sigmoid().cpu() > params['threshhold']) *1 #torch.softmax(preds, dim=1)[:, 1]
    mask = mask.detach().cpu()
    intersection, union = intersection_and_union(preds, mask)
    f1_score = calculate_f1_score(preds, mask)

  return loss, intersection, union, f1_score

Delete GPU-RAM

In [ ]:
import gc  # Garbage Collector
gc.collect()
torch.cuda.empty_cache()

In [28]:
best_iou = 0

for epoch in range(1, params['epochs']+1):
    print(f'\nEpoch: {epoch}')
    for phase in ['train', 'val']:
        total_intersection=0
        total_union=0
        total_loss=0
        total_f1=0

        pbar = tqdm(enumerate(dataloaders_dict[phase]),
                              total=len(dataloaders_dict[phase]),
                              leave=True,
                              dynamic_ncols=True)
        for i, (img, mask) in pbar:
            img, mask = img.to(params['device'], non_blocking=True), mask.to(params['device'], non_blocking=True)

            # training
            if phase == 'train':
                loss, intersection, union, f1 = train(model, img, mask)
            # validation
            else:
                loss, intersection, union, f1 = val(model, img, mask)

            # free GPU
            del img
            del mask

            total_intersection += intersection
            total_union += union
            total_loss += loss
            iou = total_intersection/total_union
            total_f1 += f1
            pbar.set_postfix({'iou': iou, 'loss': total_loss/(i+1), 'f1': total_f1/(i+1)})

        total_loss /= len(dataloaders_dict[phase])
        total_f1 /= len(dataloaders_dict[phase])

        if phase == 'train':
          scheduler.step()

        print(f'    Phase: {phase} -->  loss={total_loss} iou={iou} f1={total_f1}')

    # save best model
    if iou > best_iou:
      print(f'Saving model with iou {iou}')
      save_model(epoch, model, ckpt_path=PATH_SRC, val_iou=iou)
      best_iou = iou
    else:
      print(f'Not saving for iou {iou}')




Epoch: 1


100%|██████████| 27/27 [03:02<00:00,  6.77s/it, iou=0.357, loss=0.572, f1=0.526]

Adjusting learning rate of group 0 to 1.0005e-04.
    Phase: train -->  loss=0.5716550151507059 iou=0.35670597162592765 f1=0.5258850673317558



100%|██████████| 100/100 [01:54<00:00,  1.14s/it, iou=0.509, loss=0.569, f1=0.53]


    Phase: val -->  loss=0.568798825442791 iou=0.50897077259596 f1=0.5298050798763958
Saving model with iou 0.50897077259596

Epoch: 2


...

    Phase: val -->  loss=0.23618650045245887 iou=0.773351807572374 f1=0.7034582489674588
Not saving for iou 0.773351807572374

Epoch: 50


100%|██████████| 27/27 [01:20<00:00,  2.99s/it, iou=0.826, loss=0.114, f1=0.896]


Adjusting learning rate of group 0 to 2.2122e-04.
    Phase: train -->  loss=0.11352154364188512 iou=0.826094043302827 f1=0.8956781544971301


 39%|███▉      | 39/100 [00:42<01:09,  1.13s/it, iou=0.764, loss=0.231, f1=0.732]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
 58%|█████▊    | 58/100 [01:01<00:43,  1.04s/it, iou=0.779, loss=0.224, f1=0.727]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
100%|██████████| 100/100 [01:45<00:00,  1.05s/it, iou=0.786, loss=0.215, f1=0.742]

    Phase: val -->  loss=0.21549396131187679 iou=0.7860229577679266 f1=0.7423885815566308
Not saving for iou 0.7860229577679266


### Results
normal + kleine lr

loss=0.27545809376984837 iou=0.7928296608501525 f1=0.7245020875022741


---
Crop + 50Epoch

loss=0.21603019520640374 iou=0.8143358869038733 f1=0.7431837448744342
